# Dynamics 365 Business Central Trouble Shooting Guide (TSG) - Login issues (SaaS)

This notebook contains Kusto queries that can help getting to the root cause of a login issue for an environment in the online version of Business Central (SaaS). Each section in the notebook contains links to the TSG part of the authorization telemetry documentation in [aka.ms/bctelemetry](aka.ms/bctelemetry), as well as Kusto queries that help dive into a specific area. 

NB! The signal used in this notebook is only available in version 16.2 (or newer) of Business Central online, so check the version of your environment.

## 1\. Get setup: Load up Python libraries and connect to Application Insights

First you need to set the notebook Kernel to Python3, load the KQLmagic module (did you install it?) and connect to your Application Insights resource (get appid and appkey from the API access page in the Application Insights portal)

In [10]:
# load the KQLmagic module
%reload_ext Kqlmagic

In [11]:
# Connect to the Application Insights API
%kql appinsights://appid='<add app id from the Application Insights portal>';appkey='<add API key from the Application Insights portal>'

## 2\. Define filters

This workbook is designed for troubleshooting a single environment. Please provide values for aadTenantId and environmentName (or use a config file).

In [12]:
# Add values for AAD tenant id, environment name, and date range.
# It is possible to leave the value for environment name blank (if you want to analyze across all values of the parameter)

# You can either use configuration file (INI file format) or set filters directly. 

# If you specify a config file, then variables set here takes precedence over manually set filter variables
# config file name and directory (full path)
configFile = "c:/tmp/notebook.ini"


# Add AAD tenant id and environment name here
aadTenantId = "MyaaDtenantId"
environmentName = ""
extensionId = "MyExtensionId"

# date filters for the analysis
# use YYYY-MM-DD format for the dates (ISO 8601)
startDate = "2020-11-20"
endDate = "2020-11-24"




# Do not edit this code section
import configparser
config = configparser.ConfigParser()

config.read(configFile)

if bool(config.defaults()):
    if config.has_option('DEFAULT', 'aadTenantId'):
        aadTenantId = config['DEFAULT']['aadTenantId']
    if config.has_option('DEFAULT', 'environmentName'):
        environmentName = config['DEFAULT']['environmentName']
    if config.has_option('DEFAULT', 'extensionId'):
        extensionId = config['DEFAULT']['extensionId']
    if config.has_option('DEFAULT', 'startDate'):    
        startDate = config['DEFAULT']['startDate']
    if config.has_option('DEFAULT', 'endDate'):
        endDate = config['DEFAULT']['endDate']

print("Using these parameters for the analysis:")
print("----------------------------------------")
print("aadTenantId         " + aadTenantId)
print("environmentName     " + environmentName)
print("startDate           " + startDate)
print("endDate             " + endDate)

Using these parameters for the analysis:
----------------------------------------
aadTenantId         0f3ec54f-5f3e-432a-8c80-0eae667293e7
environmentName     
startDate           2020-11-01
endDate             2020-12-01


# Analyze the login flow
Now you can run Kusto queries to look for possible root causes for login issues.

Either click **Run All** above to run all sections, or scroll down to the type of analysis you want to do and manually run queries

Documentation of the authentication and authorization flow: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/security/security-application

Kusto sample: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql

## Authentication

Authentication in the online version of Business Central happens strictly in Azure Active Directory (AAD). Only when a user is authenticated in AAD, a session is attempted to be created in the Business Central server (NST). When dealing with login issues, check for *absense* of signal in eventIds for Authorization in the pre-open company of opening a session in the NST (eventIds RT0001 and RT0003) to determine if the issue is related to AAD (e.g. user is disabled, wrong password, failed MFA) or maybe something happening in the customer network (could be a DNS issue, or a changed firewall rule). 

**If you do not see any signal for eventIds RT0001 and RT0003, then start troubleshooting network issues first.**

Read more in the Security Guide here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/security/security-application#authentication

In [13]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId in ('RT0001', 'RT0003' )
| summarize request_count=count() by bin(timestamp, 1h) | render timechart title= 'Number of pre-open company authorization attempts in the last day'

[{'timestamp': datetime.datetime(2020, 11, 30, 8, 0, tzinfo=tzutc()), 'request_count': 3},
 {'timestamp': datetime.datetime(2020, 12, 1, 2, 0, tzinfo=tzutc()), 'request_count': 9582},
 {'timestamp': datetime.datetime(2020, 12, 1, 21, 0, tzinfo=tzutc()), 'request_count': 9965},
 {'timestamp': datetime.datetime(2020, 12, 1, 22, 0, tzinfo=tzutc()), 'request_count': 9583},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 0, tzinfo=tzutc()), 'request_count': 4612},
 {'timestamp': datetime.datetime(2020, 12, 1, 19, 0, tzinfo=tzutc()), 'request_count': 9553},
 {'timestamp': datetime.datetime(2020, 12, 1, 20, 0, tzinfo=tzutc()), 'request_count': 9309},
 {'timestamp': datetime.datetime(2020, 11, 30, 20, 0, tzinfo=tzutc()), 'request_count': 9806},
 {'timestamp': datetime.datetime(2020, 11, 30, 21, 0, tzinfo=tzutc()), 'request_count': 9728},
 {'timestamp': datetime.datetime(2020, 11, 30, 22, 0, tzinfo=tzutc()), 'request_count': 9686},
 {'timestamp': datetime.datetime(2020, 11, 30, 23, 0, tzinfo=tzutc()), 'request_count': 9616},
 {'timestamp': datetime.datetime(2020, 12, 1, 0, 0, tzinfo=tzutc()), 'request_count': 9473},
 {'timestamp': datetime.datetime(2020, 12, 1, 12, 0, tzinfo=tzutc()), 'request_count': 9810},
 {'timestamp': datetime.datetime(2020, 12, 1, 13, 0, tzinfo=tzutc()), 'request_count': 9731},
 {'timestamp': datetime.datetime(2020, 12, 1, 11, 0, tzinfo=tzutc()), 'request_count': 9720},
 {'timestamp': datetime.datetime(2020, 11, 30, 19, 0, tzinfo=tzutc()), 'request_count': 10101},
 {'timestamp': datetime.datetime(2020, 12, 1, 3, 0, tzinfo=tzutc()), 'request_count': 9467},
 {'timestamp': datetime.datetime(2020, 12, 1, 4, 0, tzinfo=tzutc()), 'request_count': 9607},
 {'timestamp': datetime.datetime(2020, 12, 1, 8, 0, tzinfo=tzutc()), 'request_count': 9823},
 {'timestamp': datetime.datetime(2020, 12, 1, 9, 0, tzinfo=tzutc()), 'request_count': 9687},
 {'timestamp': datetime.datetime(2020, 12, 1, 10, 0, tzinfo=tzutc()), 'request_count': 9725},
 {'timestamp': datetime.datetime(2020, 12, 1, 14, 0, tzinfo=tzutc()), 'request_count': 9747},
 {'timestamp': datetime.datetime(2020, 12, 1, 18, 0, tzinfo=tzutc()), 'request_count': 9557},
 {'timestamp': datetime.datetime(2020, 12, 1, 5, 0, tzinfo=tzutc()), 'request_count': 9723},
 {'timestamp': datetime.datetime(2020, 12, 1, 6, 0, tzinfo=tzutc()), 'request_count': 9690},
 {'timestamp': datetime.datetime(2020, 12, 1, 15, 0, tzinfo=tzutc()), 'request_count': 9660},
 {'timestamp': datetime.datetime(2020, 12, 1, 7, 0, tzinfo=tzutc()), 'request_count': 9741},
 {'timestamp': datetime.datetime(2020, 11, 30, 18, 0, tzinfo=tzutc()), 'request_count': 2994},
 {'timestamp': datetime.datetime(2020, 12, 1, 1, 0, tzinfo=tzutc()), 'request_count': 9333},
 {'timestamp': datetime.datetime(2020, 12, 1, 16, 0, tzinfo=tzutc()), 'request_count': 9393},
 {'timestamp': datetime.datetime(2020, 12, 1, 17, 0, tzinfo=tzutc()), 'request_count': 9612},
 {'timestamp': datetime.datetime(2020, 11, 30, 10, 0, tzinfo=tzutc()), 'request_count': 1},
 {'timestamp': datetime.datetime(2020, 11, 30, 16, 0, tzinfo=tzutc()), 'request_count': 2},
 {'timestamp': datetime.datetime(2020, 11, 30, 11, 0, tzinfo=tzutc()), 'request_count': 2}]

## Authorization failures (pre-open company)

A user can fail authorization before the open company trigger is executed for a number of different reasons:
* The user was successfully authenticated in Azure Active Directory but the user account is disabled in Business Central.
* A user successfully authenticated in Azure Active Directory but the user does not have any entitlements in Business Central (license issue)

Read more about these types of failures in the authorization signal docs here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-authorization-trace#authorizationfailedpreopencompany

Kusto query: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql

In [14]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId == 'RT0001'
| project timestamp
, guestUser = customDimensions.guestUser
, userType = customDimensions.userType
, failureReason = customDimensions.failureReason
, entitlementSetIds = customDimensions.entitlementSetIds
| order by timestamp desc
| limit 100

,timestamp,guestUser,userType,failureReason,entitlementSetIds


[]

## Authorization failures (in the open company process)

Events show up here for a number of different reasons
* The company name is invalid
* User has no permission to access the company
* The environment is locked
* The license has expired or the trial period has ended
* The user's license is not valid for use on production companies

Read more in the authorization signal docs here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-authorization-trace#authorization-failed-open-company

Kusto query: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql


In [15]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId == 'RT0002'
| project timestamp
, clientType = customDimensions.clientType
, companyName = customDimensions.companyName
, failureReason = customDimensions.failureReason
| order by timestamp desc
| limit 100

,timestamp,clientType,companyName,failureReason


[]

## Successful logins (authentication in AAD succeded, authorization in the Business Central server succeeded)

If the user can authenticate against AAD and the two authorization steps inside the Business Central server succeeds, then a session is created and the user has successfully logged in.

Read more about application security in the Security Guide here:  https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/security/security-application#authentication

Kusto query: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql

In [16]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId == 'RT0004'
| project timestamp
, clientType = customDimensions.clientType
, companyName = customDimensions.companyName
, totalTimeInMS = toreal(totimespan(customDimensions.totalTime))/10000 // totalTime is measured in ticks, divide by 10000 to get milliseconds
| order by timestamp desc
| limit 100

,timestamp,clientType,companyName,totalTimeInMS
0,2020-12-01 23:59:59.024293100+00:00,Background,"CRONUS USA, Inc.",10.0700
1,2020-12-01 23:59:58.974290400+00:00,Background,"CRONUS USA, Inc.",5.4789
2,2020-12-01 23:59:58.884285500+00:00,Background,"CRONUS USA, Inc.",5.2953
3,2020-12-01 23:59:57.868352900+00:00,Background,"CRONUS USA, Inc.",5.0337
4,2020-12-01 23:59:57.833351100+00:00,Background,"CRONUS USA, Inc.",5.4034
5,2020-12-01 23:59:57.723346300+00:00,Background,"CRONUS USA, Inc.",4.8321
6,2020-12-01 23:59:57.633347500+00:00,Background,"CRONUS USA, Inc.",5.3238
7,2020-12-01 23:59:56.529267500+00:00,Background,"CRONUS USA, Inc.",6.1617
8,2020-12-01 23:59:56.507955100+00:00,Background,"CRONUS USA, Inc.",4.9661
9,2020-12-01 23:59:56.422947400+00:00,Background,"CRONUS USA, Inc.",5.7938


[{'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 59, 24293, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 10.07},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 58, 974290, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.4789},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 58, 884285, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.2953},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 57, 868352, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.0337},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 57, 833351, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.4034},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 57, 723346, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.8321},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 57, 633347, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.3238},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 56, 529267, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 6.1617},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 56, 507955, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.9661},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 56, 422947, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.7938},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 55, 267880, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.7948},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 55, 122877, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.8274},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 54, 92241, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 6.0697},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 54, 22233, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.4595},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 53, 932236, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.1787},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 53, 812228, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.9876},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 52, 784140, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.6289},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 52, 739138, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.4829},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 52, 649134, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.6695},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 52, 429126, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.0338},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 52, 294119, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.0705},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 51, 231416, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.7937},
 {'timestamp': datetime.datetime(2020, 12, 1, 23, 59, 51, 186416, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.1743},
 {'timestamp': datetime.datetime(2020, 12, 1, 23

In [17]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId == 'RT0004'
| extend clientType = tostring( customDimensions.clientType )
| summarize count=count() by clientType, bin(timestamp, 1h)
| render timechart title= 'Number of successful logins the last day (shown by client/session type)'

[{'clientType': 'WebClient', 'timestamp': datetime.datetime(2020, 11, 30, 8, 0, tzinfo=tzutc()), 'count': 2},
 {'clientType': 'ChildSession', 'timestamp': datetime.datetime(2020, 11, 30, 8, 0, tzinfo=tzutc()), 'count': 6},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 2, 0, tzinfo=tzutc()), 'count': 9582},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 11, 30, 18, 0, tzinfo=tzutc()), 'count': 2994},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 11, 30, 19, 0, tzinfo=tzutc()), 'count': 10101},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 11, 30, 20, 0, tzinfo=tzutc()), 'count': 9806},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 11, 30, 21, 0, tzinfo=tzutc()), 'count': 9728},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 11, 30, 22, 0, tzinfo=tzutc()), 'count': 9686},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 11, 30, 23, 0, tzinfo=tzutc()), 'count': 9616},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 0, 0, tzinfo=tzutc()), 'count': 9473},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 1, 0, tzinfo=tzutc()), 'count': 9333},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 3, 0, tzinfo=tzutc()), 'count': 9467},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 4, 0, tzinfo=tzutc()), 'count': 9607},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 5, 0, tzinfo=tzutc()), 'count': 9723},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 6, 0, tzinfo=tzutc()), 'count': 9690},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 7, 0, tzinfo=tzutc()), 'count': 9741},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 15, 0, tzinfo=tzutc()), 'count': 9660},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 16, 0, tzinfo=tzutc()), 'count': 9393},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 17, 0, tzinfo=tzutc()), 'count': 9612},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 18, 0, tzinfo=tzutc()), 'count': 9557},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 19, 0, tzinfo=tzutc()), 'count': 9552},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 20, 0, tzinfo=tzutc()), 'count': 9310},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 21, 0, tzinfo=tzutc()), 'count': 9965},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 22, 0, tzinfo=tzutc()), 'count': 9583},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 23, 0, tzinfo=tzutc()), 'count': 4612},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 11, 0, tzinfo=tzutc()), 'count': 9720},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 12, 0, tzinfo=tzutc()), 'count': 9810},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 13, 0, tzinfo=tzutc()), 'count': 9731},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 8, 0, tzinfo=tzutc()), 'count': 9823},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 9, 0, tzinfo=tzutc()), 'count': 9687},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 10, 0, tzinfo=tzutc()), 'count': 9725},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 12, 1, 14, 0, tzinfo=tzutc()), 'count': 9747},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 11, 30, 10, 0, tzinfo=tzutc()), 'count': 80},
 {'clientType': 'ChildSession', 'timestamp': datetime.datetime(2020, 11, 30, 12, 0, tzinfo=tzutc()), 'count': 2},
 {'clientType': 'WebClient', 'timestamp': datetime.datetime(2020, 11, 30, 10, 0, tzinfo=tzutc()), 'count': 1},
 {'clientType': 'Child

In [18]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and customDimensions.eventId in ('RT0001', 'RT0002', 'RT0004')
| extend attemptType = case(
    customDimensions.eventId == 'RT0001', 'Failure before open company' ,
    customDimensions.eventId == 'RT0002', 'Failure in open company trigger' ,
    customDimensions.eventId == 'RT0004', 'Successful login' , 
    'Unknown reason'
)
| summarize count=count() by attemptType, bin(timestamp, 1h)
| render timechart title= 'Number of login attempts the last day (shown by success/failure)'

[{'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 11, 30, 8, 0, tzinfo=tzutc()), 'count': 8},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 2, 0, tzinfo=tzutc()), 'count': 9582},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 11, 30, 18, 0, tzinfo=tzutc()), 'count': 2994},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 11, 30, 19, 0, tzinfo=tzutc()), 'count': 10101},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 11, 30, 20, 0, tzinfo=tzutc()), 'count': 9806},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 11, 30, 21, 0, tzinfo=tzutc()), 'count': 9728},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 11, 30, 22, 0, tzinfo=tzutc()), 'count': 9686},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 11, 30, 23, 0, tzinfo=tzutc()), 'count': 9616},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 0, 0, tzinfo=tzutc()), 'count': 9473},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 1, 0, tzinfo=tzutc()), 'count': 9333},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 3, 0, tzinfo=tzutc()), 'count': 9467},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 4, 0, tzinfo=tzutc()), 'count': 9607},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 5, 0, tzinfo=tzutc()), 'count': 9723},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 6, 0, tzinfo=tzutc()), 'count': 9690},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 7, 0, tzinfo=tzutc()), 'count': 9741},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 15, 0, tzinfo=tzutc()), 'count': 9660},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 16, 0, tzinfo=tzutc()), 'count': 9393},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 17, 0, tzinfo=tzutc()), 'count': 9612},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 18, 0, tzinfo=tzutc()), 'count': 9557},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 19, 0, tzinfo=tzutc()), 'count': 9552},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 20, 0, tzinfo=tzutc()), 'count': 9310},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 21, 0, tzinfo=tzutc()), 'count': 9965},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 22, 0, tzinfo=tzutc()), 'count': 9583},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 23, 0, tzinfo=tzutc()), 'count': 4612},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 11, 0, tzinfo=tzutc()), 'count': 9720},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 12, 0, tzinfo=tzutc()), 'count': 9810},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 13, 0, tzinfo=tzutc()), 'count': 9731},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 8, 0, tzinfo=tzutc()), 'count': 9823},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 9, 0, tzinfo=tzutc()), 'count': 9687},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 11, 30, 10, 0, tzinfo=tzutc()), 'count': 82},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 11, 30, 16, 0, tzinfo=tzutc()), 'count': 4},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 14, 0, tzinfo=tzutc()), 'count': 9747},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 12, 1, 10, 0, tzinfo=tzutc()), 'count': 9725},
 {'attemptType':